In [3]:
import pandas as pd
import requests
import boto3
import ast
import datetime
import time

In [4]:
from cobinhood_api import Cobinhood
cob = Cobinhood()

print(cob.system.get_time())

2018-08-06 07:26:10 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.system fetch "get_time"


{'success': True, 'result': {'time': 1533554770855}}


In [3]:
cob.market.get_trading_pairs()

2018-07-27 07:16:56 Chriss-MacBook-Pro.local cobinhood-api[4018] DEBUG cobinhood_api.http.market fetch "get_trading_pairs"


{'success': True,
 'result': {'trading_pairs': [{'id': 'ABT-BTC',
    'base_currency_id': 'ABT',
    'quote_currency_id': 'BTC',
    'base_max_size': '2516723.629',
    'base_min_size': '50.335',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'ABT-ETH',
    'base_currency_id': 'ABT',
    'quote_currency_id': 'ETH',
    'base_max_size': '2516723.629',
    'base_min_size': '50.335',
    'quote_increment': '0.0000001',
    'margin_enabled': False},
   {'id': 'ACT-BTC',
    'base_currency_id': 'ACT',
    'quote_currency_id': 'BTC',
    'base_max_size': '11934470.211',
    'base_min_size': '238.69',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'BAT-BTC',
    'base_currency_id': 'BAT',
    'quote_currency_id': 'BTC',
    'base_max_size': '2991280.418',
    'base_min_size': '59.826',
    'quote_increment': '0.00000001',
    'margin_enabled': False},
   {'id': 'BAT-ETH',
    'base_currency_id': 'BAT',
    'quote_currency_id': 'ETH',

In [5]:
# Instantiate resources
try:
    boto_session = boto3.Session(profile_name='loidsig')
except:
    boto_session = boto3.Session()
s3_resource = boto_session.resource('s3')
s3_bucket = 'loidsig-crypto'

sm_client = boto_session.client(
    service_name='secretsmanager',
    region_name='us-east-1',
    endpoint_url='https://secretsmanager.us-east-1.amazonaws.com'
)
get_secret_value_response = sm_client.get_secret_value(SecretId='Loidsig_CPM_Cobinhood')
key, value = ast.literal_eval(get_secret_value_response['SecretString']).popitem()
cob = Cobinhood(API_TOKEN=value)

In [6]:
print(cob.wallet.get_balances())

2018-08-06 07:26:17 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.wallet fetch "get_balances"


{'success': True, 'result': {'balances': [{'currency': 'ETH', 'type': 'exchange', 'total': '1.49', 'on_order': '0', 'locked': False, 'usd_value': '608.92212404328', 'btc_value': '0.0871283013'}]}}


In [22]:
cob_candles_list = cob.chart.get_candles(trading_pair_id='ETH-USDT', start_time=1519862400000, end_time=1519948800000, timeframe='1m')['result']['candles']
df = pd.DataFrame(cob_candles_list)
df.rename(columns={'trading_pair_id':'coin','timestamp':'close_timestamp'}, inplace=True)
df['coin'] = df['coin'].replace(regex=True, to_replace=r'-', value=r'')
df['close_timestamp_trim'] = df['close_timestamp']/1000
df['close_datetime'] = pd.to_datetime(df['close_timestamp_trim'], unit='s')

print(f"""Current: {datetime.datetime.utcfromtimestamp(time.time()).strftime('%Y-%m-%d %I:%M:%S')}
Start: {datetime.datetime.utcfromtimestamp(1519516800).strftime('%Y-%m-%d %I:%M:%S')}""")
df

2018-07-27 07:56:21 Chriss-MacBook-Pro.local cobinhood-api[4018] DEBUG cobinhood_api.http.chart fetch "get_candles"


Current: 2018-07-27 11:56:22
Start: 2018-02-25 12:00:00


,close,high,low,open,timeframe,close_timestamp,coin,volume,close_timestamp_trim,close_datetime
0,479.19,479.19,477.77,477.77,1m,1532606160000,ETHUSDT,0.09,1.532606e+09,2018-07-26 11:56:00
1,479.19,479.19,479.19,479.19,1m,1532606220000,ETHUSDT,0,1.532606e+09,2018-07-26 11:57:00
2,478.81,478.81,477.5,477.5,1m,1532606280000,ETHUSDT,2.91145321290685,1.532606e+09,2018-07-26 11:58:00
3,478.81,478.81,478.81,478.81,1m,1532606340000,ETHUSDT,0,1.532606e+09,2018-07-26 11:59:00
4,478.02,478.34,478.02,478.34,1m,1532606400000,ETHUSDT,2.263513507483162,1.532606e+09,2018-07-26 12:00:00
5,477.51,477.51,477.51,477.51,1m,1532606460000,ETHUSDT,0.048,1.532606e+09,2018-07-26 12:01:00
6,479.14,479.14,479.14,479.14,1m,1532606520000,ETHUSDT,0.04783671,1.532607e+09,2018-07-26 12:02:00
7,479.14,479.14,479.14,479.14,1m,1532606580000,ETHUSDT,0,1.532607e+09,2018-07-26 12:03:00
8,479.14,479.14,479.14,479.14,1m,1532606640000,ETHUSDT,0,1.532607e+09,2018-07-26 12:04:00
9,479.14,479.14,479.14,479.14,1m,1532606700000,ETHUSDT,0,1.532607e+09,2018-07-26 12:05:00


In [21]:
df[df['volume'] != '0']

,close,high,low,open,timeframe,close_timestamp,coin,volume,close_timestamp_trim,close_datetime
132,879.9,879.9,870,870,1m,1519870320000,ETHUSDT,1.12,1.519870e+09,2018-03-01 02:12:00
156,899.9,899.9,879.9,879.9,1m,1519871760000,ETHUSDT,0.6469286,1.519872e+09,2018-03-01 02:36:00
191,899.9,899.9,899.9,899.9,1m,1519873860000,ETHUSDT,0.31932385,1.519874e+09,2018-03-01 03:11:00
315,840,855.1,840,855.1,1m,1519881300000,ETHUSDT,1,1.519881e+09,2018-03-01 05:15:00
321,838.15,840,838.15,840,1m,1519881660000,ETHUSDT,1,1.519882e+09,2018-03-01 05:21:00
324,825,838.15,825,838.15,1m,1519881840000,ETHUSDT,1.48,1.519882e+09,2018-03-01 05:24:00
647,825,836.1,825,836.1,1m,1519901220000,ETHUSDT,1,1.519901e+09,2018-03-01 10:47:00
880,875,875,875,875,1m,1519915200000,ETHUSDT,0.05991693,1.519915e+09,2018-03-01 14:40:00
881,898.8,898.8,898.8,898.8,1m,1519915260000,ETHUSDT,0.04792709,1.519915e+09,2018-03-01 14:41:00
901,840,840,840,840,1m,1519916460000,ETHUSDT,0.098,1.519916e+09,2018-03-01 15:01:00


In [2]:
'eth-usdt'.replace('-', '')

'ethusdt'

### Placing an order

In [20]:
data = {
    "trading_pair_id": "ETH-USDT",
    "side": "bid",
    "type": "market",
    "size": "1.000"
}
result = cob.trading.post_orders(data)

2018-08-06 08:02:53 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.trading fetch "post_orders"


In [21]:
trade_id = result['result']['order']['id']
trade_id

'7b03bbf4-e67c-44ea-878c-11fe822462f0'

In [22]:
cob.trading.get_orders(trade_id)

2018-08-06 08:02:59 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.trading fetch "get_orders"


{'success': True,
 'result': {'order': {'id': '7b03bbf4-e67c-44ea-878c-11fe822462f0',
   'trading_pair_id': 'ETH-USDT',
   'side': 'bid',
   'type': 'market',
   'price': '0',
   'size': '1',
   'filled': '1',
   'state': 'filled',
   'timestamp': 1533556974069,
   'eq_price': '408.0537288676',
   'completed_at': '2018-08-06T12:02:54.095305855Z',
   'source': 'exchange'}}}

In [23]:
print(cob.wallet.get_balances())

2018-08-06 08:03:14 Chriss-MacBook-Pro.local cobinhood-api[1204] DEBUG cobinhood_api.http.wallet fetch "get_balances"


{'success': True, 'result': {'balances': [{'currency': 'USDT', 'type': 'exchange', 'total': '2.4501111324', 'on_order': '0', 'locked': False, 'usd_value': '2.4511962633444842022', 'btc_value': '0.000351002920827624'}, {'currency': 'ETH', 'type': 'exchange', 'total': '1.48', 'on_order': '0', 'locked': False, 'usd_value': '606.16495162776', 'btc_value': '0.0867979744'}]}}
